In [1]:
import gym
import seagul.envs
import numpy as np
from numpy import pi, sin, cos
import time
import torch
import matplotlib.pyplot as plt

import numpy as np
from numpy import sin, cos, pi
from control import lqr

# dynamics from our gym environment
I2 = .083; I1 = .083 
l1 = 1; l2 = 2
m1 = 1; m2 = 1
lc1 = .5; lc2 = 1
g = 9.8
th1 = pi; th2 = 0
th1d = 0; th2d = 0

M = np.array([[I2 + I2 + m2*l1**2 + 2*m2*l1*lc2*cos(th2), I2 + m2*l1*lc2*cos(th2)], [I2 + m2*l1*lc2*cos(th2), I2]])
C = np.array([[-2*m2*l1*lc2*sin(th2)*th2d, -m2*l1*lc2*sin(th2)*th2d], [-m2*l1*lc2*sin(th2)*th2d, 0]])
G = np.array([[-m1*g*lc1*sin(th1) - m2*g*(l1*sin(th1) + lc2*sin(th1 + th2))],[-m2*g*lc2*sin(th1+ th2)]])
B = np.array([[0,1]])

In [2]:
Alin= np.array([[0,0, 1, 0],[0, 0, 0, 1], [g*(m1*l1 + m2*l1 + m2*lc2), (m2*g*lc2), 0, 0 ],[m2*g*lc2, m2*g*lc2, 0, 0]])
Bl = np.linalg.inv(M)*B
Blin = np.array([[0,0],[0,0],Bl[0,:], Bl[1,:]])

In [3]:
Q = np.identity(4)
Q[0,0] = .1**2
Q[1,1] = .1**2
Q[2,2] = 10**2
Q[3,3] = 10**2
R = np.identity(2)**.01
K,S,E = lqr(Alin,Blin,Q,R)

In [4]:
k = K[1,:]
#k[1] = -k[1]
print(k)
#k = np.array([-242, 96, -100, -49])
k = np.array([-1000, 1000, -10, -10])

[-2448.4953532  -983.2360222  -427.4614525  -160.2030707]


In [6]:
env = gym.make('su_acrobot-v0')
env.num_steps=5000

#model.gate_fn.net_fn = gate
action_hist = np.zeros((env.num_steps,1))
action_debug = np.zeros((env.num_steps,4))

state_hist = np.zeros((env.num_steps, env.observation_space.shape[0]))
reward_hist = np.zeros((env.num_steps, 1))

obs = env.reset()
env.state[0] = np.pi
env.state[1] = .2
env.state[2] = 0
env.state[3] = 0

for i in range(env.num_steps):
        #actions, _, _, _ = model.step(obs)
        #actions = np.zeros(1)
        #actions = env.action_space.sample()
        env.render()
        goal = np.copy(env.state)
        goal[0] -= pi
        #import pdb; pdb.set_trace()
        actions = -k.dot(goal)
        action_debug[i,:] = -k*goal
        
        obs, reward, done, _ = env.step(actions)        
        action_hist[i,:] = np.copy(actions)
        state_hist[i,:] = np.copy(obs)
        reward_hist[i,:] = np.copy(reward)
        #time.sleep(.25)
        if done:
            break
            
            
plt.plot(action_hist)
plt.title('action')

plt.figure()
plt.plot(action_debug)
plt.title('reward')
plt.legend(['0','1','2','3'])

plt.figure()
plt.plot(state_hist[:,0])
plt.title("cos(th1)")

plt.figure()
plt.plot(state_hist[:,1])
plt.title("sin(th1)")

plt.figure()
plt.plot(state_hist[:,2])
plt.title("cos(th2)")

plt.figure()
plt.plot(state_hist[:,3])
plt.title("sin(th2)")

plt.figure()
plt.plot(state_hist[:,4])
plt.title("th1 dot")

plt.figure()
plt.plot(state_hist[:,5])
plt.title("th2 dot")


print(i)
print(sum(reward_hist))

KeyboardInterrupt: 